In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Creacion de dataset y obtencion de datos


In [ ]:
from pandas.core.indexing import item_from_zerodim
import os,glob
import pandas as pd
import json,re

# Ruta de la carpeta principal
ruta_carpeta_principal = '/content/drive/MyDrive/PeerRead-master/data'
lista_part=["dev/parsed_pdfs/","test/parsed_pdfs/","train/parsed_pdfs/"]
lista_conf=["arxiv.cs.ai_2007-2017"]
rutas_archivos_json = []
secc = {}

def limpiar_texto(texto):
    #texto = re.sub(r'\n\d+\n', '\n', texto)
    #texto = re.sub(r'[×/]', '', texto)
    #texto = re.sub(r'\b\d+\b', '', texto)
    #texto = re.sub(r'\n+', '\n', texto).strip()
    # Reemplazar caracteres problemáticos con un espacio en blanco
    #texto_limpio = ''.join(char if ord(char) < 128 else ' ' for char in texto)
    #return texto_limpio
    # Reemplazar caracteres problemáticos con un espacio en blanco
    texto_limpio = ''.join(char if char.isprintable() else ' ' for char in texto)
    return texto_limpio


intro_pattern = re.compile(r'(?:\d*\.?\s*)?(introduction)', re.IGNORECASE)
conclusion_pattern = re.compile(r'(?:\d*\.?\s*)?(conclusion)', re.IGNORECASE)

for confe in lista_conf:
  print(confe)
  for particion in lista_part:
    #print(particion)
    ruta=(ruta_carpeta_principal+"/"+confe+"/"+particion)
    #print("Explorando ruta:", ruta)
    archivos=glob.glob(ruta+"/*.json")
    #print(archivos)
    particion_sin_ruta = particion.strip('/').split('/')[0]

    for archivo in archivos:
      with open(archivo, 'r') as f:
        contenido = json.load(f)
        #print(contenido.keys())
        nombre=contenido["name"]
        diccontent=contenido["metadata"]
        #print(diccontent)
        t = diccontent["title"]
        #print(t)
        e = diccontent["emails"]
        #print(e)
        autores=diccontent["authors"]
        #print(autores)
        dicsecciones=diccontent["sections"]

        referencias_archivo = diccontent["references"]
        c = 0
        abstract= diccontent["abstractText"]
        #print(dicsecciones)
        cadena=""
        introduccion=""
        conclusion=""
        contenido=""


        #print(dicsecciones)
        if dicsecciones!=None:

          for item in dicsecciones:
            if item['heading'] == None:
                cadena = ""
                text=""
            else:
                cadena = cadena + item['heading'] + "\n"
                cadena = limpiar_texto(cadena) + ""

                # Verificar si el heading coincide con el patron de introduccion y concllu
                if re.search(intro_pattern, item['heading']):
                  introduccion = limpiar_texto(item['text'])
                elif re.search(conclusion_pattern, item['heading']):
                  conclusion = limpiar_texto(item['text'])
                else:
                  text= limpiar_texto(item['text'])
                  contenido += f"**{item['heading']}**\n"+ text+ "\n"

            texto= item['text'] #re.sub(r'(\n\d+)+\n', '',item['text'])
            texto = limpiar_texto(texto)
            cadena=cadena+texto+"\n"

          rutas_archivos_json.append({
                      "Conferencia":confe,
                      "Particion":particion_sin_ruta,
                      "Archivo": nombre,
                      "Title": t,
                      "Emails": e,
                      "Autores": autores,
                      "Texto Completo": cadena,
                      "Abstract": abstract,
                      "Introducción": introduccion,
                      "Contenido": contenido,
                      "Conclusion": conclusion,
                      "Referencias": referencias_archivo
                  })

        else:
          print("Archivo "+nombre +" sin contenido")

          rutas_archivos_json.append({
                      "Conferencia":confe,
                      "Particion":particion_sin_ruta,
                      "Archivo": nombre,
                      "Title": t,
                      "Emails": e,
                      "Autores": autores,
                      "Texto Completo": "",
                      "Abstract": abstract,
                      "Introducción": "",
                      "Contenido": "",
                      "Conclusion": "",
                      "Referencias": referencias_archivo
                  })


df = pd.DataFrame(rutas_archivos_json)
df.to_csv(ruta_carpeta_principal+"/"+"arxivAi_primer_ciclo.csv", sep=',', index=False, encoding='utf-8-sig')
print("Datos guardados en CSV:", ruta_carpeta_principal+"/"+"arxivAi_primer_ciclo.csv")



arxiv.cs.ai_2007-2017
Archivo 1302.1525.pdf sin contenido
Archivo 1302.6791.pdf sin contenido
Archivo 1302.1557.pdf sin contenido
Archivo 1302.1574.pdf sin contenido
Archivo 1206.5266.pdf sin contenido
Archivo 1301.2296.pdf sin contenido
Archivo 1301.2289.pdf sin contenido
Archivo 1302.1540.pdf sin contenido
Archivo 1301.2295.pdf sin contenido
Archivo 1301.2312.pdf sin contenido
Archivo 1206.5294.pdf sin contenido
Archivo 1301.2305.pdf sin contenido
Archivo 1206.5249.pdf sin contenido
Archivo 1302.1551.pdf sin contenido
Archivo 1302.6823.pdf sin contenido
Archivo 1302.6802.pdf sin contenido
Archivo 1302.6826.pdf sin contenido
Archivo 1301.2281.pdf sin contenido
Archivo 1302.6815.pdf sin contenido
Archivo 1302.1559.pdf sin contenido
Archivo 1301.2306.pdf sin contenido
Archivo 1301.2313.pdf sin contenido
Archivo 1302.1523.pdf sin contenido
Archivo 1301.2256.pdf sin contenido
Archivo 1708.06816.pdf sin contenido
Archivo 1302.6818.pdf sin contenido
Archivo 1301.2287.pdf sin contenido
Archi

# obtener estado de aceptacion

In [ ]:
import os
import json
import pandas as pd

ruta_carpeta_principal = '/content/drive/MyDrive/PeerRead-master/data'
lista_part = ["dev/reviews/", "test/reviews/", "train/reviews/"]
lista_conf = ["arxiv.cs.ai_2007-2017"]
csv_path = os.path.join(ruta_carpeta_principal, "reviews_arxivAi.csv")

# Crear o cargar el DataFrame desde el archivo CSV existente
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
else:
    df = pd.DataFrame(columns=["Archivo", "Estado"])

dataframes = []  # Lista para almacenar los DataFrames temporales

for confe in lista_conf:
    for particion in lista_part:
        ruta = os.path.join(ruta_carpeta_principal, confe, particion)
        archivos = os.listdir(ruta)

        for archivo in archivos:
            archivo_path = os.path.join(ruta, archivo)

            with open(archivo_path, 'r') as f:
                contenido = json.load(f)
                id = contenido["id"]
                id = id+'.pdf'
                accepted = contenido["accepted"]

                # Verificar si accepted es un booleano y asignar 1 o 0 en consecuencia
                if isinstance(accepted, bool):
                    accepted_value = "1" if accepted else "0"
                else:
                    accepted_value = "1" if accepted.lower() == "verdadero" else "0"

                temp_df = pd.DataFrame({"Archivo": [id], "Accepted": [accepted_value]})
                dataframes.append(temp_df)

# Concatenar todos los DataFrames temporales en uno solo
if dataframes:
    df = pd.concat(dataframes, ignore_index=True)

# Guardar el DataFrame actualizado en el archivo CSV
df.to_csv(csv_path, sep=',', index=False, encoding='utf-8')
print("Datos guardados en CSV:", csv_path)


Datos guardados en CSV: /content/drive/MyDrive/PeerRead-master/data/reviews_arxivAi.csv


#combinar csvs

In [ ]:
import pandas as pd

# Especifica la ruta de tus archivos CSV en tu Google Drive
ruta_carpeta_principal = '/content/drive/MyDrive/PeerRead-master/data'

archivo1 = os.path.join(ruta_carpeta_principal, "reviews_arxivAi.csv")
archivo2 = os.path.join(ruta_carpeta_principal, "arxivAi_primer_ciclo.csv")

# Cargar archivos CSV en DataFrames
df1 = pd.read_csv(archivo1)
df2 = pd.read_csv(archivo2)

df1 = df1.drop_duplicates(subset='Title', keep='first')
df2 = df2.drop_duplicates(subset='Title', keep='first')

# Union con base a la columna 'Archivo'
result = pd.merge(df1, df2, on='Archivo', how='outer')
#result = result.dropna(subset=['Conferencia'])
print(result)

result.to_csv('/content/drive/MyDrive/arxivAi_datos_completos.csv', index=False)

ParserError: ignored

In [ ]:
import os
import pandas as pd

# Especifica la ruta de tus archivos CSV en tu Google Drive
ruta_carpeta_principal = '/content/drive/MyDrive/PeerRead-master/data'

archivo1 = os.path.join(ruta_carpeta_principal, "arxivAi_primer_ciclo.csv")
archivo2 = os.path.join(ruta_carpeta_principal, "reviews_arxivAi.csv")

# Carga los archivos CSV en DataFrames
df1 = pd.read_csv(archivo1)
df2 = pd.read_csv(archivo2)

# Guarda el número inicial de filas en cada DataFrame
num_filas_inicial_df1 = len(df1)
print("Filas del doc 1:",num_filas_inicial_df1)
num_filas_inicial_df2 = len(df2)
print("Filas del doc 2:",num_filas_inicial_df2)

# Union de csv con base en la columna 'Title'
result = pd.merge(df1, df2, on='Archivo', how='outer')
print("Número total de elementos al unir:", len(result))

# Dropear filas con valores nulos en columnas
result = result.dropna(subset=['Accepted'])
#result = result.dropna(subset=['Contenido'])
print("Número total de elementos despues de dropear:", len(result))
# Calcular la diferencia en el numero de filas despues de la limpieza


#print(result)
# Guarda el resultado en un CSV
result.to_csv('/content/drive/MyDrive/arxivAi_datos_completos.csv', index=False)
print(result)

# Imprime el numero total de filas eliminadas
#print("Número total de filas eliminadas:", num_total_filas_eliminadas)



Filas del doc 1: 4092
Filas del doc 2: 4092
Número total de elementos al unir: 4092
Número total de elementos despues de dropear: 4092
                Conferencia Particion         Archivo  \
0     arxiv.cs.ai_2007-2017       dev   1302.4942.pdf   
1     arxiv.cs.ai_2007-2017       dev  1703.04912.pdf   
2     arxiv.cs.ai_2007-2017       dev   1201.2430.pdf   
3     arxiv.cs.ai_2007-2017       dev   1305.3321.pdf   
4     arxiv.cs.ai_2007-2017       dev  1511.03532.pdf   
...                     ...       ...             ...   
4087  arxiv.cs.ai_2007-2017     train   1306.0386.pdf   
4088  arxiv.cs.ai_2007-2017     train  1605.04056.pdf   
4089  arxiv.cs.ai_2007-2017     train  1603.03518.pdf   
4090  arxiv.cs.ai_2007-2017     train   1406.3124.pdf   
4091  arxiv.cs.ai_2007-2017     train  1612.07837.pdf   

                                                  Title  \
0     Implementation of Continuous Bayesian Networks...   
1                                                   NaN   
2  